In [ ]:
import streamlit as st
import pandas as pd
import zipfile
import io
import os

# Title & Sidebar
st.sidebar.header('Unzip Summary of Episode Note (USENT)')

# Upload Meta File
uploaded_file = st.sidebar.file_uploader("Choose a Meta File", type=['csv'])

bar = st.progress(0)
df = None  # Placeholder for the dataframe
file_names = []  # Placeholder for file names

if uploaded_file is not None:
    # Only execute this part when Meta File is uploaded
    df = pd.read_csv(uploaded_file)
    st.header("Meta File Data")
    st.title("_Data Upload Completed_ :sunglasses:")
    st.text(f'Data Frame Shape : {df.shape}')
    st.write(df.head())

    loinc_file = pd.crosstab(df['Organization'], df['LOINC'])
    st.write(loinc_file.head())
    bar.progress(20)

else:
    # Ensure no other code executes until a file is uploaded
    st.warning("Please upload a Meta File to proceed.")

# Upload ZIP file (This code will not execute unless explicitly uploaded)
uploaded_zip = st.sidebar.file_uploader("Choose a Zipped File", type=["zip"])

if uploaded_zip is not None:
    zip_bytes = io.BytesIO(uploaded_zip.read())

    with zipfile.ZipFile(zip_bytes, "r") as zip_ref:
        file_list = zip_ref.namelist()

        # Filter only .csv files and get base file names
        excel_files = [f for f in file_list if f.endswith(".csv")]
        file_names = [os.path.basename(f) for f in excel_files]

#         st.subheader("CSV Files in ZIP:")
#         st.text(f"Total CSV Files Found: {len(file_names)}")
#         for f in file_names:
#             st.markdown(f"- {f}")

    bar.progress(40)

# Proceed if Meta file is uploaded and filters are chosen
if df is not None:

    # Sidebar Filters (This will execute only if Meta file is uploaded)
    loinc_options = ["Select All"] + sorted(df['LOINC'].dropna().unique())
    selected_loinc = st.sidebar.selectbox("Filter by LOINC", loinc_options)

    org_options = ["Select All"] + sorted(df['Organization'].dropna().unique())
    selected_org = st.sidebar.selectbox("Filter by Organization", org_options)

    # Start with full data
    filtered_df = df.copy()

    # Apply LOINC filter
    if selected_loinc != "Select All":
        filtered_df = filtered_df[filtered_df["LOINC"] == selected_loinc]

    # Show filtered df after LOINC filter
    st.subheader(f"Filtered Data (LOINC: {selected_loinc})")
    st.write(filtered_df)


    # Apply Organization filter
    if selected_org != "Select All":
        filtered_df_by_loinc_org = filtered_df[filtered_df["Organization"] == selected_org]
    #         st.write(filtered_df_by_loinc_org)

        # Show filtered df after both filters
        st.subheader(f"Filtered Data (LOINC: {selected_loinc}, Organization: {selected_org})")
        st.write(filtered_df_by_loinc_org)
        st.subheader("Matching File Names from ZIP:")
        st.write(filtered_df_by_loinc_org['File_Name'].to_list())

#     # Match file names from ZIP
#     if uploaded_zip is not None:
#         filtered_loinc_codes = filtered_df_by_loinc_org["LOINC"].unique()

#         # Match files using filtered LOINC codes
#         if selected_org == "Select All":
#             matching_files = [f for f in file_names if any(code in f for code in filtered_loinc_codes)]
#         else:
#             matching_files = [f for f in file_names if selected_org in f]

#         st.subheader("Matching File Names from ZIP:")
#         st.write(matching_files)

#         bar.progress(70)
